<a href="https://colab.research.google.com/github/mrasw/Big-Data/blob/main/EAS_Company_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mengambil file dari s3

In [ ]:
!wget https://bayueas-114.s3.amazonaws.com/data-source/classification-dataset-v1.csv -O classification-dataset-v1.csv

--2022-06-10 12:18:29--  https://bayueas-114.s3.amazonaws.com/data-source/classification-dataset-v1.csv
Resolving bayueas-114.s3.amazonaws.com (bayueas-114.s3.amazonaws.com)... 52.217.97.244
Connecting to bayueas-114.s3.amazonaws.com (bayueas-114.s3.amazonaws.com)|52.217.97.244|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 399850400 (381M) [text/csv]
Saving to: ‘classification-dataset-v1.csv’

classification-data 100%[===================>] 381.33M  73.5MB/s    in 5.2s    

2022-06-10 12:18:35 (73.2 MB/s) - ‘classification-dataset-v1.csv’ saved [399850400/399850400]



install pyspark

In [ ]:
# Install Apache Spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


membuat spark session

In [ ]:
from pyspark.sql import *

spark = SparkSession.builder.appName('Clustering using K-Means').getOrCreate()


read csv file

In [ ]:
df=spark.read.option("header", True).csv("classification-dataset-v1.csv")

df.printSchema()

root
 |-- Category: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- homepage_text: string (nullable = true)
 |-- h1: string (nullable = true)
 |-- h2: string (nullable = true)
 |-- h3: string (nullable = true)
 |-- nav_link_text: string (nullable = true)
 |-- meta_keywords: string (nullable = true)
 |-- meta_description: string (nullable = true)



show file content

In [ ]:
df.head()
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|                null|                null|electricians vero...|                null|
| vero beach electric| commercial light...|         electrician| Indian River County|           Sebastian|          Ft Pierce"|Providing quality...|    

show count of row and column

In [ ]:
print((df.count(), len(df.columns)))

(250419, 10)


count null value in each columns

In [ ]:
from pyspark.sql.functions import *

# col_null_cnt_df =  df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns])

# display(col_null_cnt_df)


show null in "Category" Column

In [ ]:
from pyspark.sql import functions as F
df.where(F.col("Category").isNull()).show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    null|    blue chip stocks|tax saving mutual...|          debt funds|     sip mutual fund|best investment o...|best performing m...|systematic invest...|    best investments|best mutual funds...|
|    null|         elss scheme|mutual fund compa...|      top elss funds| sip investment plan| tax on mutual funds|best equity mutua...|   top 10 elss funds|high return mutua...|top mutual funds ...|


show null value in "Website" column

In [ ]:
df.where(F.col("website").isNull()).show()

+--------------------+-------+--------------------+-------------+--------------------+----+----+-------------+-------------+----------------+
|            Category|website|        company_name|homepage_text|                  h1|  h2|  h3|nav_link_text|meta_keywords|meta_description|
+--------------------+-------+--------------------+-------------+--------------------+----+----+-------------+-------------+----------------+
|ET EXPERTISE#sep#...|   null|,en,à propos,nos ...|         null|TRIOX Urgences En...|null|null|         null|         null|            null|
|big and small.#se...|   null|                null|         null|                null|null|null|         null|         null|            null|
|in salt lake for ...|   null|                null|         null|                null|null|null|         null|         null|            null|
|  doesnât stop ...|   null|                null|         null|                null|null|null|         null|         null|            null|
|  the

show null value in "company_name" column

In [ ]:
df.where(F.col("company_name").isNull()).show()

+--------------------+--------------------+------------+--------------------+--------------------+----+----+-------------+-------------+----------------+
|            Category|             website|company_name|       homepage_text|                  h1|  h2|  h3|nav_link_text|meta_keywords|meta_description|
+--------------------+--------------------+------------+--------------------+--------------------+----+----+-------------+-------------+----------------+
|       661.663.0300"|Get To Know Elias...|        null|Elias Medical bak...|For the best valu...|null|null|         null|         null|            null|
|    Locally owned...| we have everythi...|        null|                null|                null|null|null|         null|         null|            null|
|             Anytime| anywhere.#sep#Ca...|        null|                null|                null|null|null|         null|         null|            null|
|big and small.#se...|                null|        null|                null

show null value in "homepage_text" column

In [ ]:
df.where(F.col("homepage_text").isNull()).show()

+--------------------+--------------------+--------------------+-------------+--------------------+----+----+-------------+-------------+----------------+
|            Category|             website|        company_name|homepage_text|                  h1|  h2|  h3|nav_link_text|meta_keywords|meta_description|
+--------------------+--------------------+--------------------+-------------+--------------------+----+----+-------------+-------------+----------------+
|661.663.0300   Fa...| Suite 203 Bakers...| CA 93311 | (phon...|         null|                null|null|null|         null|         null|            null|
|    (fax) 661.663...|Offering Bakersfi...|Welcome to ELIAS ...|         null|                null|null|null|         null|         null|            null|
|ET EXPERTISE#sep#...|                null|,en,à propos,nos ...|         null|TRIOX Urgences En...|null|null|         null|         null|            null|
|    Locally owned...| we have everythi...|                null|      

show null value in "h1" column

In [ ]:
df.where(F.col("h1").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|null|                null|                null|                null|electricians vero...|                null|
|          Healthcare|    eliasmedical.com|       elias medical|    site map | en...|null|                null|                null|                null|                null|                null|
|661.663.0300   Fa..

show null value in "h2" column

In [ ]:
df.where(F.col("h2").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|null|                null|                null|electricians vero...|                null|
|          Healthcare|    eliasmedical.com|       elias medical|    site map | en...|                null|null|                null|                null|                null|                null|
|661.663.0300   Fa..

show null value in "h3" column

In [ ]:
df.where(F.col("h3").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|  h3|nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|null|         null|electricians vero...|                null|
|          Healthcare|    eliasmedical.com|       elias medical|    site map | en...|                null|                null|null|         null|                null|                null|
|661.663.0300   Fa...| Suite 203 Bakers...| CA 93311 | 

show null value in "nav_link_text" column

In [ ]:
df.where(F.col("nav_link_text").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|                null|         null|electricians vero...|                null|
| vero beach electric| commercial light...|         electrician| Indian River County|           Sebastian|          Ft Pierce"|Providing quality...|         null|                nu

show null value in "meta_keywords" column

In [ ]:
df.where(F.col("meta_keywords").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
| vero beach electric| commercial light...|         electrician| Indian River County|           Sebastian|          Ft Pierce"|Providing quality...|                null|         null|                null|
|          Healthcare|    eliasmedical.com|       elias medical|    site map | en...|                null|                null|                null|                null|         nu

show null value in "meta_description" column

In [ ]:
df.where(F.col("meta_description").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|                null|                null|electricians vero...|            null|
| vero beach electric| commercial light...|         electrician| Indian River County|           Sebastian|          Ft Pierce"|Providing quality...|                null

membuat kolom baru dari gabungan h1 h2 h3 menjadi heading



In [ ]:
# concat_ws(",","firstname",'lastname'
new_df = df.withColumn("headings",concat_ws("","h1",'h2',"h3"))
new_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|            headings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|                null|                null|electricians vero...|                null|                    |
| vero beach electric| commercial light...|         electrician| Ind

drop h1 h2 h3

In [ ]:
cols = ("h1","h2","h3")
new_df1 = new_df.drop(*cols)
new_df.printSchema()
# new_df.drop(*cols).printSchema()
new_df.show()

root
 |-- Category: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- homepage_text: string (nullable = true)
 |-- h1: string (nullable = true)
 |-- h2: string (nullable = true)
 |-- h3: string (nullable = true)
 |-- nav_link_text: string (nullable = true)
 |-- meta_keywords: string (nullable = true)
 |-- meta_description: string (nullable = true)
 |-- headings: string (nullable = false)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|            headings|
+--------------------+--------------------+--------------------+---------------

In [ ]:
print((new_df1.count(), len(new_df1.columns)))
new_df1.printSchema()

(250419, 8)
root
 |-- Category: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- homepage_text: string (nullable = true)
 |-- nav_link_text: string (nullable = true)
 |-- meta_keywords: string (nullable = true)
 |-- meta_description: string (nullable = true)
 |-- headings: string (nullable = false)



In [ ]:
# new_df1.filter("state IS NULL AND gender IS NULL").show()
new_df1.filter(new_df1.Category.isNull()).show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Category|             website|        company_name|       homepage_text|       nav_link_text|       meta_keywords|    meta_description|            headings|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    null|    blue chip stocks|tax saving mutual...|          debt funds|systematic invest...|    best investments|best mutual funds...|sip mutual fundbe...|
|    null|         elss scheme|mutual fund compa...|      top elss funds|   top 10 elss funds|high return mutua...|top mutual funds ...|sip investment pl...|
|    null|    health insurance|            children|              doctor|doctor's office n...|     allergic asthma|         baby doctor| doctors near me ...|
|    null|                null|                null|

In [ ]:
# from pyspark.sql.functions import *
new_df2 = new_df1.dropna()

In [ ]:
new_df2 = new_df2.withColumn("text",concat_ws(" ","homepage_text",'nav_link_text',"meta_keywords","meta_description","headings"))

In [ ]:
new_df2.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|       nav_link_text|       meta_keywords|    meta_description|            headings|                text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|           Materials| woodgreentimber.com|wood green timber...|"      Wood Green...| in stock and rea...| however we are e...| and or milled to...| London and our P...|"      Wood Green...|
|           Materials|     houstonchem.com|    houston chemical|"              Ho...| Chemicals and Pi...| we have clients ...| LLC.  All Rights...| BUILD A DOOR""  ...|"              Ho...|
|           Materials|   metalprideinc.com|  

In [ ]:
cols = ("homepage_text","nav_link_text","meta_keywords","meta_description","headings")
new_df3 = new_df2.drop(*cols)
new_df2.printSchema()
new_df3.printSchema()
# new_df.drop(*cols).printSchema()
new_df3.show()

root
 |-- Category: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- homepage_text: string (nullable = true)
 |-- nav_link_text: string (nullable = true)
 |-- meta_keywords: string (nullable = true)
 |-- meta_description: string (nullable = true)
 |-- headings: string (nullable = false)
 |-- text: string (nullable = false)

root
 |-- Category: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- text: string (nullable = false)

+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|                text|
+--------------------+--------------------+--------------------+--------------------+
|           Materials| woodgreentimber.com|wood green timber...|"      Wood Green...|
|           Materials|     houstonchem.com|    houston chemical|"              Ho...|
|           Ma